# Decision Analysis Project 2

- Mateusz Tabaszewski 151945
- Bartłomiej Pukacki 151942

This is our notebook dedicated to the second project from decision analysis classes, in this notebook we set out to implement our own versions of UTA and AHP methods used for multi-criteria decision analysis (MCDA). 

Sections present in this notebook:
* Imports - imports used for the notebook
* Utilities - general functions and classes which can be used outside the implemented methods
* Data - showcase of the constructed dataset
* UTA - implementation of UTA method
* AHP - implementation of AHP method
* Use - comparison of methods including PROMETHEE II results

## Imports

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)

## Utilities

In [36]:
def get_pairwise_comparisons(data, column, boundaries):
    """
    Returns pairwise comparisons based on specified boundaries
    
    Parameters:
        - data: dataframe with data
        - column: name of the column for which the comparison will be performed
        - boundaries: list of 8 boundaries of differences for which the Saaty's scale preference intensity increases
            e.g. with list [1,2,3,4,5,6,10,12] Saaty's preference of 9 applied for difference in values >12,
            the preference of 8 is applied for differences in interval (10, 12], 1 for [0, 1] etc.
    """
    proportions = []
    # Apply comparison based on boundaries
    for i in range(len(data)):
        row = []
        for j in range(len(data)):
            if data.at[j, column] != 0:
                diff = data.at[i, column] - data.at[j, column]
                if diff < 0:
                    row.append("L")
                elif diff <= boundaries[0]:
                    row.append(1)
                elif diff <= boundaries[1]:
                    row.append(2)
                elif diff <= boundaries[2]:
                    row.append(3)
                elif diff <= boundaries[3]:
                    row.append(4)
                elif diff <= boundaries[4]:
                    row.append(5)
                elif diff <= boundaries[5]:
                    row.append(6)
                elif diff <= boundaries[6]:
                    row.append(7)
                elif diff <= boundaries[7]:
                    row.append(8)
                else:
                    row.append(9)

            else:
                row.append(None)
        proportions.append(row)

    # Create inverse values
    for i in range(len(data)):
        for j in range(len(data)):
            if proportions[i][j] == "L":
                proportions[i][j] = round(1/proportions[j][i], 2)

    df_proportions = pd.DataFrame(proportions, columns=data['id'], index=data['id'])
    return df_proportions

## Data


id | name                          | price | critic_score | user_score | length | genres   | num_of_achievements |
---|-------------------------------|-------|--------------|------------|--------|----------|---------------------|
0  | Dark Souls: Remastered        | 150   | 84           | 83         | 44     | 11       | 41                  |
1  | Dark Souls III                | 200   | 89           | 90         | 49     | 11       | 43                  |
2  | Terraria                      | 46    | 81           | 81         | 102    | 9        | 115                 |
3  | Baldur's Gate 3               | 250   | 96           | 89         | 107    | 8        | 54                  |
4  | Dave the Diver                | 92    | 90           | 83         | 32     | 7        | 43                  |
5  | Rust                          | 153   | 69           | 65         | 37     | 10       | 92                  |
6  | Hollow Knight                 | 68    | 90           | 91         | 42     | 10       | 63                  |
7  | Portal 2                      | 46    | 95           | 89         | 14     | 7        | 51                  |
8  | Vampire Survivors             | 20    | 86           | 83         | 25     | 9        | 204                 |
9  | Hades                         | 115   | 93           | 88         | 49     | 9        | 49                  |
10 | Subnautica                    | 139   | 87           | 86         | 43     | 5        | 17                  |
11 | Dishonored                    | 45    | 88           | 83         | 18     | 8        | 80                  |
12 | Ori and the Will of the Wisps | 108   | 90           | 89         | 16     | 10       | 37                  |
13 | Inside                        | 72    | 93           | 83         | 4      | 6        | 14                  |
14 | The Forest                    | 72    | 83           | 75         | 28     | 9        | 45                  |
15 | Skyrim                        | 90    | 96           | 86         | 114    | 11       | 75                  |
16 | Teardown                      | 120   | 80           | 81         | 22     | 6        | 27                  |
17 | Dying Light                   | 90    | 74           | 81         | 36     | 5        | 78                  |
18 | Enter the Gungeon             | 68    | 82           | 80         | 62     | 4        | 54                  |
19 | Payday 3                      | 169   | 66           | 31         | 10     | 5        | 22                  |
20 | Kao the Kangaroo              | 129   | 65           | 75         | 8      | 8        | 26                  |
21 | Assassin's Creed Unity        | 120   | 72           | 56         | 35     | 11       | 57                  |
22 | Trials Fusion                 | 80    | 79           | 71         | 23     | 2        | 51                  |
23 | The Sims 3                    | 28    | 83           | 78         | 78     | 3        | 65                  |
24 | Titan Souls                   | 68    | 74           | 61         | 4      | 8        | 27                  |

In [3]:
data = pd.read_csv("data.csv")
data

,id,name,price,critic_score,user_score,length,genres,num_of_achievements
0,0,Dark Souls: Remastered,150,84,83,44,11,41
1,1,Dark Souls III,200,89,90,49,11,43
2,2,Terraria,46,81,81,102,9,115
3,3,Baldur's Gate 3,250,96,89,107,8,54
4,4,Dave the Diver,92,90,83,32,7,43
5,5,Rust,153,69,65,37,10,92
6,6,Hollow Knight,68,90,91,42,10,63
7,7,Portal 2,46,95,89,14,7,51
8,8,Vampire Survivors,20,86,83,25,9,204
9,9,Hades,115,93,88,49,9,49


## UTA

## AHP

### Criteria hierarchy:

Goal: buying a video game that is described by quality and quantity measures

```tree
goal
├── quality
│   ├── user_score
│   ├── critic_score
│   └── genres
└── quantity
    ├── price
    ├── length
    └── num_of_achievements
```

### Pairwise comparisons of hierarchy elements

| user_score |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|    1 | 5    | 1    | 6    | 1    | 5    | 9    | 1    | 1    | 5    | 1    | 2    | 5    | 1    | 5    | 8    | 2    | 6    | 6    | 7    |    9 | 8    | 9    | 9    | 8    | 9    |
|    2 | 1    | 0.17 | 1    | 0.17 | 1    | 9    | 0.14 | 0.17 | 1    | 0.2  | 0.33 | 1    | 0.17 | 1    | 4    | 0.33 | 1    | 1    | 1    |    9 | 4    | 9    | 7    | 2    | 9    |
|    3 | 4    | 1    | 6    | 1    | 4    | 9    | 1    | 1    | 4    | 1    | 2    | 4    | 1    | 4    | 8    | 2    | 6    | 6    | 6    |    9 | 8    | 9    | 9    | 8    | 9    |
|    4 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|    5 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 1    | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.14 | 0.11 | 0.11 | 0.11 | 0.12 |    9 | 0.14 | 6    | 0.25 | 0.12 | 2    |
|    6 | 6    | 1    | 7    | 1    | 6    | 9    | 1    | 1    | 6    | 2    | 3    | 6    | 1    | 6    | 9    | 3    | 7    | 7    | 8    |    9 | 9    | 9    | 9    | 8    | 9    |
|    7 | 4    | 1    | 6    | 1    | 4    | 9    | 1    | 1    | 4    | 1    | 2    | 4    | 1    | 4    | 8    | 2    | 6    | 6    | 6    |    9 | 8    | 9    | 9    | 8    | 9    |
|    8 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|    9 | 3    | 1    | 5    | 1    | 3    | 9    | 0.5  | 1    | 3    | 1    | 1    | 3    | 1    | 3    | 8    | 1    | 5    | 5    | 6    |    9 | 8    | 9    | 9    | 7    | 9    |
|   10 | 2    | 0.5  | 3    | 0.5  | 2    | 9    | 0.33 | 0.5  | 2    | 1    | 1    | 2    | 0.5  | 2    | 8    | 1    | 3    | 3    | 4    |    9 | 8    | 9    | 8    | 6    | 9    |
|   11 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|   12 | 4    | 1    | 6    | 1    | 4    | 9    | 1    | 1    | 4    | 1    | 2    | 4    | 1    | 4    | 8    | 2    | 6    | 6    | 6    |    9 | 8    | 9    | 9    | 8    | 9    |
|   13 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|   14 | 0.17 | 0.12 | 0.25 | 0.12 | 0.17 | 7    | 0.11 | 0.12 | 0.17 | 0.12 | 0.12 | 0.17 | 0.12 | 0.17 | 1    | 0.12 | 0.25 | 0.25 | 0.33 |    9 | 1    | 9    | 2    | 0.5  | 8    |
|   15 | 2    | 0.5  | 3    | 0.5  | 2    | 9    | 0.33 | 0.5  | 2    | 1    | 1    | 2    | 0.5  | 2    | 8    | 1    | 3    | 3    | 4    |    9 | 8    | 9    | 8    | 6    | 9    |
|   16 | 1    | 0.17 | 1    | 0.17 | 1    | 9    | 0.14 | 0.17 | 1    | 0.2  | 0.33 | 1    | 0.17 | 1    | 4    | 0.33 | 1    | 1    | 1    |    9 | 4    | 9    | 7    | 2    | 9    |
|   17 | 1    | 0.17 | 1    | 0.17 | 1    | 9    | 0.14 | 0.17 | 1    | 0.2  | 0.33 | 1    | 0.17 | 1    | 4    | 0.33 | 1    | 1    | 1    |    9 | 4    | 9    | 7    | 2    | 9    |
|   18 | 0.5  | 0.14 | 1    | 0.17 | 0.5  | 8    | 0.12 | 0.17 | 0.5  | 0.17 | 0.25 | 0.5  | 0.17 | 0.5  | 3    | 0.25 | 1    | 1    | 1    |    9 | 3    | 9    | 6    | 1    | 9    |
|   19 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 |    1 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 |
|   20 | 0.17 | 0.12 | 0.25 | 0.12 | 0.17 | 7    | 0.11 | 0.12 | 0.17 | 0.12 | 0.12 | 0.17 | 0.12 | 0.17 | 1    | 0.12 | 0.25 | 0.25 | 0.33 |    9 | 1    | 9    | 2    | 0.5  | 8    |
|   21 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.17 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 |    9 | 0.11 | 1    | 0.12 | 0.11 | 0.33 |
|   22 | 0.12 | 0.11 | 0.14 | 0.11 | 0.12 | 4    | 0.11 | 0.11 | 0.12 | 0.11 | 0.12 | 0.12 | 0.11 | 0.12 | 0.5  | 0.12 | 0.14 | 0.14 | 0.17 |    9 | 0.5  | 8    | 1    | 0.2  | 7    |
|   23 | 0.33 | 0.12 | 0.5  | 0.12 | 0.33 | 8    | 0.12 | 0.12 | 0.33 | 0.14 | 0.17 | 0.33 | 0.12 | 0.33 | 2    | 0.17 | 0.5  | 0.5  | 1    |    9 | 2    | 9    | 5    | 1    | 9    |
|   24 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.5  | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.11 | 0.12 | 0.11 | 0.11 | 0.11 | 0.11 |    9 | 0.12 | 3    | 0.14 | 0.11 | 1    |

In [23]:
data[["id", "user_score"]]

,id,user_score
0,0,83
1,1,90
2,2,81
3,3,89
4,4,83
5,5,65
6,6,91
7,7,89
8,8,83
9,9,88


In [39]:
boundaries = [2,4,5,6,7,9,10,15]
comparisons = get_pairwise_comparisons(data, "user_score", boundaries)
comparisons

id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
id,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.00,0.20,1.00,0.25,1.00,9.00,0.17,0.25,1.00,0.33,0.50,1.00,0.25,1.00,6.00,0.50,1.00,1.00,2.00,9,6.00,9.00,8.00,3.00,9.00
1,5.00,1.00,6.00,1.00,5.00,9.00,1.00,1.00,5.00,1.00,2.00,5.00,1.00,5.00,8.00,2.00,6.00,6.00,7.00,9,8.00,9.00,9.00,8.00,9.00
2,1.00,0.17,1.00,0.17,1.00,9.00,0.14,0.17,1.00,0.20,0.33,1.00,0.17,1.00,4.00,0.33,1.00,1.00,1.00,9,4.00,9.00,7.00,2.00,9.00
3,4.00,1.00,6.00,1.00,4.00,9.00,1.00,1.00,4.00,1.00,2.00,4.00,1.00,4.00,8.00,2.00,6.00,6.00,6.00,9,8.00,9.00,9.00,8.00,9.00
4,1.00,0.20,1.00,0.25,1.00,9.00,0.17,0.25,1.00,0.33,0.50,1.00,0.25,1.00,6.00,0.50,1.00,1.00,2.00,9,6.00,9.00,8.00,3.00,9.00
5,0.11,0.11,0.11,0.11,0.11,1.00,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.14,0.11,0.11,0.11,0.12,9,0.14,6.00,0.25,0.12,2.00
6,6.00,1.00,7.00,1.00,6.00,9.00,1.00,1.00,6.00,2.00,3.00,6.00,1.00,6.00,9.00,3.00,7.00,7.00,8.00,9,9.00,9.00,9.00,8.00,9.00
7,4.00,1.00,6.00,1.00,4.00,9.00,1.00,1.00,4.00,1.00,2.00,4.00,1.00,4.00,8.00,2.00,6.00,6.00,6.00,9,8.00,9.00,9.00,8.00,9.00
8,1.00,0.20,1.00,0.25,1.00,9.00,0.17,0.25,1.00,0.33,0.50,1.00,0.25,1.00,6.00,0.50,1.00,1.00,2.00,9,6.00,9.00,8.00,3.00,9.00


In [43]:
print(comparisons.to_markdown())

|   id |    0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |    8 |    9 |   10 |   11 |   12 |   13 |   14 |   15 |   16 |   17 |   18 |   19 |   20 |   21 |   22 |   23 |   24 |
|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
|    0 | 1    | 0.2  | 1    | 0.25 | 1    | 9    | 0.17 | 0.25 | 1    | 0.33 | 0.5  | 1    | 0.25 | 1    | 6    | 0.5  | 1    | 1    | 2    |    9 | 6    | 9    | 8    | 3    | 9    |
|    1 | 5    | 1    | 6    | 1    | 5    | 9    | 1    | 1    | 5    | 1    | 2    | 5    | 1    | 5    | 8    | 2    | 6    | 6    | 7    |    9 | 8    | 9    | 9    | 8    | 9    |
|    2 | 1    | 0.17 | 1    | 0.17 | 1    | 9    | 0.14 | 0.17 | 1    | 0.2  | 0.33 | 1    | 0.17 | 1    | 4    | 0.33 | 1    | 1    | 1    |    9 | 4    | 9    | 7    | 2    | 9    |
|    3 | 4    | 1    | 6    | 1    | 4    | 9    | 1    | 1    | 4    | 1    | 2

## TESTS